In [3]:
## 1-2. 지역별 그룹핑 + 폐지된건 개칭된걸로 변경

# 필요한 패키지 불러오기
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

sido_code_map = {
    '11': '서울특별시',
    '26': '부산광역시',
    '27': '대구광역시',
    '28': '인천광역시',
    '29': '광주광역시',
    '30': '대전광역시',
    '31': '울산광역시',
    '36': '세종특별자치시',
    '41': '경기도',
    '42': '강원특별자치도',               # 폐지
    '43': '충청북도',
    '44': '충청남도',
    '45': '전북특별자치도',             # 폐지
    '46': '전라남도',
    '47': '경상북도',
    '48': '경상남도',
    '49': '제주특별자치도',              # 폐지, 데이터 내 미존재
    '50': '제주특별자치도',      # 2006년 1월 이후 제주도의 승격·개칭 
    '51': '강원특별자치도',      # 2023년 7월 이후 강원도의 승격·개칭 
    '52': '전북특별자치도'       # 2023년 7월 이후 전라북도의 승격·개칭
}

# 데이터 경로 설정
data_dir = r'C:\Git_project\SAS-Korea_Data-Analysis\SAS-Korea_Data-Analysis\data'

# TRAIN_DATA와 TEST_DATA 로드
train_data = pd.read_csv(os.path.join(data_dir, 'TRAIN_DATA.csv'), encoding='cp949')
test_data = pd.read_csv(os.path.join(data_dir, 'TEST_DATA.csv'), encoding='cp949')
기온_data = pd.read_csv(os.path.join(data_dir, '기온real.csv'))  # 기온 데이터 로드

# 지역 코드 전처리 및 지역명 매핑 - 훈련 데이터
train_data['DIST_CD1'] = train_data['DIST_CD'].astype(str).str[:2]
train_data['SIDO_NM'] = train_data['DIST_CD1'].map(sido_code_map)

# 지역 코드 전처리 및 지역명 매핑 - 테스트 데이터
test_data['DIST_CD1'] = test_data['DIST_CD'].astype(str).str[:2]
test_data['SIDO_NM'] = test_data['DIST_CD1'].map(sido_code_map)

# 기온 데이터 전처리
기온_data['DATA_YM'] = pd.to_datetime(기온_data['DATA_YM']).dt.strftime('%Y%m')

# 날짜 타입 통일
train_data['DATA_YM'] = train_data['DATA_YM'].astype(str)
test_data['DATA_YM'] = test_data['DATA_YM'].astype(str)
기온_data['DATA_YM'] = 기온_data['DATA_YM'].astype(str)

# 공백 제거
train_data['SIDO_NM'] = train_data['SIDO_NM'].str.strip()
test_data['SIDO_NM'] = test_data['SIDO_NM'].str.strip()
기온_data['SIDO_NM'] = 기온_data['SIDO_NM'].str.strip()

# 병합 - 훈련 데이터
train_merged = pd.merge(train_data, 기온_data, on=['DATA_YM', 'SIDO_NM'], how='left')

# 병합 - 테스트 데이터
test_merged = pd.merge(test_data, 기온_data, on=['DATA_YM', 'SIDO_NM'], how='left')

# DIST_CD1과 SIDO_NM 컬럼 제거
train_merged = train_merged.drop(['DIST_CD1', 'SIDO_NM'], axis=1)
test_merged = test_merged.drop(['DIST_CD1', 'SIDO_NM'], axis=1)

# 결과 확인
print(f"훈련 데이터 병합 결과: {train_merged.shape}")
print(f"테스트 데이터 병합 결과: {test_merged.shape}")

# 병합된 데이터를 CSV 파일로 저장
train_merged.to_csv(os.path.join(data_dir, 'TRAIN_DATA_ver2.csv'), index=False, encoding='cp949')
test_merged.to_csv(os.path.join(data_dir, 'TEST_DATA_ver2.csv'), index=False, encoding='cp949')

print("병합된 데이터가 TRAIN_DATA_ver2.csv와 TEST_DATA_ver2.csv로 저장되었습니다.")

훈련 데이터 병합 결과: (26099, 27)
테스트 데이터 병합 결과: (6525, 26)
병합된 데이터가 TRAIN_DATA_ver2.csv와 TEST_DATA_ver2.csv로 저장되었습니다.


In [7]:
# PCA 데이터 로드
pca_data = pd.read_csv(os.path.join(data_dir, 'pca.csv'))

# PCA 데이터의 DATA_YM 형식 변환 (date 형식 -> 'YYYYMM' 형식)
if pd.api.types.is_datetime64_any_dtype(pca_data['DATA_YM']):
    # 날짜 형식인 경우 YYYYMM 형식으로 변환
    pca_data['DATA_YM'] = pd.to_datetime(pca_data['DATA_YM']).dt.strftime('%Y%m')
else:
    # 이미 문자열인 경우 형식 확인 후 필요시 변환
    try:
        pca_data['DATA_YM'] = pd.to_datetime(pca_data['DATA_YM']).dt.strftime('%Y%m')
    except:
        print("DATA_YM 형식 변환 중 오류가 발생했습니다. 형식을 확인해주세요.")

# 데이터 타입 통일
pca_data['DATA_YM'] = pca_data['DATA_YM'].astype(str)
pca_data['AREA_ID'] = pca_data['AREA_ID'].astype(str)

# TRAIN_DATA_ver2와 TEST_DATA_ver2 다시 로드
train_ver2 = pd.read_csv(os.path.join(data_dir, 'TRAIN_DATA_ver2.csv'), encoding='cp949')
test_ver2 = pd.read_csv(os.path.join(data_dir, 'TEST_DATA_ver2.csv'), encoding='cp949')

# 데이터 타입 통일
train_ver2['DATA_YM'] = train_ver2['DATA_YM'].astype(str)
train_ver2['AREA_ID'] = train_ver2['AREA_ID'].astype(str)
test_ver2['DATA_YM'] = test_ver2['DATA_YM'].astype(str)
test_ver2['AREA_ID'] = test_ver2['AREA_ID'].astype(str)

# PCA 데이터에서 필요한 컬럼만 선택
pca_selected = pca_data[['AREA_ID', 'DATA_YM', 'PC1', 'PC2', 'CLUSTER']]

# 병합 - 훈련 데이터
train_ver2_pca = pd.merge(train_ver2, pca_selected, on=['AREA_ID', 'DATA_YM'], how='left')

# 병합 - 테스트 데이터
test_ver2_pca = pd.merge(test_ver2, pca_selected, on=['AREA_ID', 'DATA_YM'], how='left')

# 결측치 확인 (병합 전)
print(f"PCA 병합 전 훈련 데이터 크기: {train_ver2.shape}")
print(f"PCA 병합 전 테스트 데이터 크기: {test_ver2.shape}")

# 결측치 확인 (병합 후)
print(f"PCA 병합 후 훈련 데이터 크기: {train_ver2_pca.shape}")
print(f"PCA 병합 후 테스트 데이터 크기: {test_ver2_pca.shape}")

print("\nPCA 변수 결측치 확인 - 병합 직후:")
print("훈련 데이터:", train_ver2_pca[['PC1', 'PC2', 'CLUSTER']].isnull().sum())
print("테스트 데이터:", test_ver2_pca[['PC1', 'PC2', 'CLUSTER']].isnull().sum())

# AREA_ID별 평균값 계산
pca_means = pca_data.groupby('AREA_ID')[['PC1', 'PC2']].mean().reset_index()
# CLUSTER는 범주형이므로 최빈값 사용
pca_mode = pca_data.groupby('AREA_ID')['CLUSTER'].agg(lambda x: x.mode()[0]).reset_index()

# 결측치 채우기 - 훈련 데이터
for idx, row in train_ver2_pca[train_ver2_pca['PC1'].isnull()].iterrows():
    area_id = row['AREA_ID']
    # PC1, PC2 평균값으로 채우기
    if area_id in pca_means['AREA_ID'].values:
        mean_values = pca_means[pca_means['AREA_ID'] == area_id].iloc[0]
        train_ver2_pca.at[idx, 'PC1'] = mean_values['PC1']
        train_ver2_pca.at[idx, 'PC2'] = mean_values['PC2']
    
    # CLUSTER 최빈값으로 채우기
    if area_id in pca_mode['AREA_ID'].values:
        mode_value = pca_mode[pca_mode['AREA_ID'] == area_id].iloc[0]['CLUSTER']
        train_ver2_pca.at[idx, 'CLUSTER'] = mode_value

# 결측치 채우기 - 테스트 데이터
for idx, row in test_ver2_pca[test_ver2_pca['PC1'].isnull()].iterrows():
    area_id = row['AREA_ID']
    # PC1, PC2 평균값으로 채우기
    if area_id in pca_means['AREA_ID'].values:
        mean_values = pca_means[pca_means['AREA_ID'] == area_id].iloc[0]
        test_ver2_pca.at[idx, 'PC1'] = mean_values['PC1']
        test_ver2_pca.at[idx, 'PC2'] = mean_values['PC2']
    
    # CLUSTER 최빈값으로 채우기
    if area_id in pca_mode['AREA_ID'].values:
        mode_value = pca_mode[pca_mode['AREA_ID'] == area_id].iloc[0]['CLUSTER']
        test_ver2_pca.at[idx, 'CLUSTER'] = mode_value

# 결측치 확인 (채운 후)
print("\nPCA 변수 결측치 확인 - AREA_ID별 평균/최빈값으로 채운 후:")
print("훈련 데이터:", train_ver2_pca[['PC1', 'PC2', 'CLUSTER']].isnull().sum())
print("테스트 데이터:", test_ver2_pca[['PC1', 'PC2', 'CLUSTER']].isnull().sum())

# 여전히 남아있는 결측치가 있다면 전체 평균/최빈값으로 채우기
if train_ver2_pca['PC1'].isnull().sum() > 0 or test_ver2_pca['PC1'].isnull().sum() > 0:
    # 전체 평균 계산
    total_pc1_mean = pca_data['PC1'].mean()
    total_pc2_mean = pca_data['PC2'].mean()
    total_cluster_mode = pca_data['CLUSTER'].mode()[0]
    
    # 남은 결측치 채우기
    train_ver2_pca['PC1'].fillna(total_pc1_mean, inplace=True)
    train_ver2_pca['PC2'].fillna(total_pc2_mean, inplace=True)
    train_ver2_pca['CLUSTER'].fillna(total_cluster_mode, inplace=True)
    
    test_ver2_pca['PC1'].fillna(total_pc1_mean, inplace=True)
    test_ver2_pca['PC2'].fillna(total_pc2_mean, inplace=True)
    test_ver2_pca['CLUSTER'].fillna(total_cluster_mode, inplace=True)
    
    print("\nPCA 변수 결측치 확인 - 전체 평균/최빈값으로 채운 후:")
    print("훈련 데이터:", train_ver2_pca[['PC1', 'PC2', 'CLUSTER']].isnull().sum())
    print("테스트 데이터:", test_ver2_pca[['PC1', 'PC2', 'CLUSTER']].isnull().sum())

# 병합된 데이터를 CSV 파일로 저장
train_ver2_pca.to_csv(os.path.join(data_dir, 'TRAIN_DATA_ver4.csv'), index=False, encoding='cp949')
test_ver2_pca.to_csv(os.path.join(data_dir, 'TEST_DATA_ver4.csv'), index=False, encoding='cp949')

print("\nPCA 변수가 추가된 데이터가 TRAIN_DATA_ver3.csv와 TEST_DATA_ver3.csv로 저장되었습니다.")


PCA 병합 전 훈련 데이터 크기: (26099, 27)
PCA 병합 전 테스트 데이터 크기: (6525, 26)
PCA 병합 후 훈련 데이터 크기: (32041, 30)
PCA 병합 후 테스트 데이터 크기: (6962, 29)

PCA 변수 결측치 확인 - 병합 직후:
훈련 데이터: PC1        4
PC2        4
CLUSTER    4
dtype: int64
테스트 데이터: PC1        5102
PC2        5102
CLUSTER    5102
dtype: int64

PCA 변수 결측치 확인 - AREA_ID별 평균/최빈값으로 채운 후:
훈련 데이터: PC1        0
PC2        0
CLUSTER    0
dtype: int64
테스트 데이터: PC1        0
PC2        0
CLUSTER    0
dtype: int64

PCA 변수가 추가된 데이터가 TRAIN_DATA_ver3.csv와 TEST_DATA_ver3.csv로 저장되었습니다.
